## Deliverable 2. Create a Customer Travel Destinations Map.

In [21]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weatherpy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Upernavik,GL,72.7868,-56.1549,33.78,98,85,5.53,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,71.51,57,65,4.56,broken clouds
2,2,Albany,US,42.6001,-73.9662,63.18,50,42,7.00,scattered clouds
3,3,Asfi,MA,32.2994,-9.2372,64.67,88,24,6.93,few clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,29.86,93,75,23.02,light snow


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
10,10,Butaritari,KI,3.0707,172.7902,83.91,79,96,18.28,light rain
12,12,Aden,YE,12.7794,45.0367,86.41,70,99,7.92,overcast clouds
14,14,Salinas,US,36.6777,-121.6555,81.57,68,90,14.97,overcast clouds
15,15,Westport,US,41.1415,-73.3579,75.74,51,1,1.99,clear sky
18,18,Leo,BF,11.1000,-2.1000,87.64,44,98,8.90,overcast clouds
19,19,Avera,US,33.1940,-82.5271,87.53,47,1,7.96,clear sky
20,20,Korla,CN,41.7597,86.1469,78.53,14,5,0.49,clear sky
25,25,Sinnamary,GF,5.3833,-52.9500,76.01,93,100,8.05,light rain
26,26,Meulaboh,ID,4.1363,96.1285,78.69,83,90,2.95,overcast clouds
27,27,Kapaa,US,22.0752,-159.3190,82.40,65,20,10.36,few clouds


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
10,10,Butaritari,KI,3.0707,172.7902,83.91,79,96,18.28,light rain
12,12,Aden,YE,12.7794,45.0367,86.41,70,99,7.92,overcast clouds
14,14,Salinas,US,36.6777,-121.6555,81.57,68,90,14.97,overcast clouds
15,15,Westport,US,41.1415,-73.3579,75.74,51,1,1.99,clear sky
18,18,Leo,BF,11.1000,-2.1000,87.64,44,98,8.90,overcast clouds
19,19,Avera,US,33.1940,-82.5271,87.53,47,1,7.96,clear sky
20,20,Korla,CN,41.7597,86.1469,78.53,14,5,0.49,clear sky
25,25,Sinnamary,GF,5.3833,-52.9500,76.01,93,100,8.05,light rain
26,26,Meulaboh,ID,4.1363,96.1285,78.69,83,90,2.95,overcast clouds
27,27,Kapaa,US,22.0752,-159.3190,82.40,65,20,10.36,few clouds


In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Butaritari,KI,83.91,light rain,3.0707,172.7902,
12,Aden,YE,86.41,overcast clouds,12.7794,45.0367,
14,Salinas,US,81.57,overcast clouds,36.6777,-121.6555,
15,Westport,US,75.74,clear sky,41.1415,-73.3579,
18,Leo,BF,87.64,overcast clouds,11.1000,-2.1000,
19,Avera,US,87.53,clear sky,33.1940,-82.5271,
20,Korla,CN,78.53,clear sky,41.7597,86.1469,
25,Sinnamary,GF,76.01,light rain,5.3833,-52.9500,
26,Meulaboh,ID,78.69,overcast clouds,4.1363,96.1285,
27,Kapaa,US,82.40,few clouds,22.0752,-159.3190,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")
        

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [23]:
# 7. Drop the rows where there is no Hotel Name.
clean = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = clean.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Butaritari,KI,83.91,light rain,3.0707,172.7902,Isles Sunset Lodge
12,Aden,YE,86.41,overcast clouds,12.7794,45.0367,Sama Emirate Hotel
14,Salinas,US,81.57,overcast clouds,36.6777,-121.6555,Laurel Inn & Conference Center
15,Westport,US,75.74,clear sky,41.1415,-73.3579,The Westport Inn
18,Leo,BF,87.64,overcast clouds,11.1000,-2.1000,Hotel Sissilis
20,Korla,CN,78.53,clear sky,41.7597,86.1469,Gulisitan Hotel
25,Sinnamary,GF,76.01,light rain,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
26,Meulaboh,ID,78.69,overcast clouds,4.1363,96.1285,Kost Pria
27,Kapaa,US,82.40,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
29,Turbat,PK,88.84,clear sky,26.0023,63.0440,Mubashar Quarters


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "weatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 30.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))